### Get Started

In [1]:
import os
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [27]:
DIR = os.path.dirname(os.path.abspath(__name__))
DB_PATH = os.path.join(DIR, 'db/')
DB_PATH

'/home/sanchez/VScode_consolidated/task1/db/'

In [28]:
# Connect to the running ChromaDB instance
client = chromadb.PersistentClient(
  path=DB_PATH,
  settings=Settings(),
  tenant=DEFAULT_TENANT,
  database=DEFAULT_DATABASE,)


In [ ]:
# Create or get a collection
collection = client.create_collection("weka")

In [51]:
# Add data to the collection
documents = ["This is a beach", "The forest is green", "Urban areas are crowded"]
metadatas = [{"category": "beach"}, {"category": "forest"}, {"category": "urban"}]
ids = ["1", "2", "3"]

collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [46]:
# Query the collection
results = collection.query(
    query_texts=["weka supporting dataset formats"],
    n_results=3  # Number of results to return
)

# Print the results
print(results)

{'ids': [['2', '3', '4']], 'distances': [[0.8691518582313926, 0.9016820837823974, 0.9688559958091476]], 'metadatas': [[None, None, None]], 'embeddings': None, 'documents': [['the default setting of 16 to 64MB is usually too small. If you get error s that\nclasses are not found, check your CLASSPATH : does it include weka.jar ? You\ncan explicitly set CLASSPATH via the-cpcommand line option as well.\nWe will begin by describing basic concepts and ideas. Then, we will desc ribe\ntheweka.filters package, which is used to transform input data, e.g. for\npreprocessing, transformation, feature generation and so on.', 'Then we will focus on the machine learning algorithms themselves. The se\nare called Classiﬁers in WEKA. We will restrict ourselves to common set tings\nfor all classiﬁers and shortly note representatives for all main app roaches in\nmachine learning.\nAfterwards, practical examples are given.\nFinally, in the docdirectory of WEKA you ﬁnd a documentation of all java\nclasses wi

### SentenceTransformer

In [53]:
from sentence_transformers import SentenceTransformer

/home/sanchez/VScode_consolidated/task1/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [54]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [60]:
DOG_embeddings = model.encode(["DOG"])
dog_embeddings = model.encode(["dog"])
(DOG_embeddings == dog_embeddings).all()

np.True_

In [62]:
good_embeddings = model.encode(["good"])
better_embeddings = model.encode(["better"])
best_embeddings = model.encode(["best"])
print(
  (good_embeddings == better_embeddings).all(),
  (good_embeddings == best_embeddings).all(),
  (better_embeddings == best_embeddings).all()
)

False False False


In [64]:
very_good_embeddings = model.encode(["very good"])
better_embeddings_embeddings = model.encode(["better"])
print((very_good_embeddings == better_embeddings_embeddings).all())

False


### Add Langchain

In [79]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [80]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [81]:
vector_store = Chroma(
    collection_name="documnet_collection",
    embedding_function=embeddings,
    persist_directory="db/",
)

In [82]:
# Add documents
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)


In [161]:
document_11 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_12 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_13 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

In [163]:
documents = [
    document_11,
    document_12,
    document_13,
]

# uuids = [str(uuid4()) for _ in range(len(documents))]
ids = [11, 12, 13]
vector_store.add_documents(documents=documents)

['0638bc76-8c5d-48e4-8c75-c3fcaf38478f',
 '33fa3b82-e01c-4313-8707-b97cb2db4044',
 '3c48a076-f043-4e4e-b663-7a1406080471']

### upload to database

In [1]:
import os
DIR = os.path.dirname(os.path.abspath(__name__))
DB_PATH = os.path.join(DIR, 'db/')

from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
import chromadb
persistent_client = chromadb.PersistentClient(
    path=DB_PATH,
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,)

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/sanchez/VScode_consolidated/task1/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain_chroma import Chroma
collection_name = "weka"
vector_store = Chroma(
  client=persistent_client,
  collection_name=collection_name,
  embedding_function=embeddings,
)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=500,
  chunk_overlap=50,
  length_function=len,
  keep_separator=False,
  add_start_index=True,
  is_separator_regex=False,
  separators=['\n\n', '\n', ' ', '']
)

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

In [6]:
# Split the large document into small documnets
async def split_document(Document:Document):
  return Document.metadata, splitter.create_documents([Document.page_content])
  # return splitter.split_documents([text])

In [8]:
from typing import List
# Split text into chunks
async def split_text(input:str|List[str]):
  if isinstance(input,str):
    return splitter.create_documents([input])
  if isinstance(input,List[str]):
    return splitter.create_documents(input)
  return ''

In [9]:
# Load the pdf file
async def load_pdf(pdf_file):
  # Initialize the PDF loader inside the function
  loader = PyPDFLoader(
    pdf_file,
    extract_images=True,
    headers=None,
    extraction_mode="plain",
  )
  pacge_count = 1
  async for page in loader.alazy_load():
    document = Document(
      metadata=page.metadata,
      page_content=page.page_content
    )
    print("page:",pacge_count)
    pacge_count += 1

    yield await split_document(document) # Yield each chunk one at a time 

In [10]:
async def upload_pdf(pdf_file:str):
  ids = list()
  id_calibration = 1
  async for metadata, chunks in load_pdf(pdf_file):
    for id, chunk in enumerate(chunks):
      chunk.metadata.update(metadata)
      ids.append(id_calibration+id)

    results = vector_store.add_documents(documents=chunks)
    print(len(results))
    # break
    id_calibration = ids[-1]+1
    ids = list()

In [14]:
async def main():
  await upload_pdf("WekaManual_13to15.pdf")
await main()

page: 1
4
page: 2
5
page: 3
3


In [17]:
results = vector_store.similarity_search(
  "what are weka supported dataset formats?", k=1)
print(results)

[Document(metadata={'page': 2, 'source': 'WekaManual_13to15.pdf', 'start_index': 889}, page_content='the main() routine of weka.core.Instances :\njava weka.core.Instances data/soybean.arff\nweka.core oﬀers some other useful routines, e.g. converters.C45Loader and\nconverters.CSVLoader ,whichcanbeusedtoimportC45datasetsandcomma/tab-\nseparated datasets respectively, e.g.:\njava weka.core.converters.CSVLoader data.csv > data.arf f\njava weka.core.converters.C45Loader c45_filestem > data .arff')]
